In [1]:
from sklearn.datasets import load_iris

# iris 데이터셋 로딩

X, y = load_iris(return_X_y=True)

print(X[:5])

print(y[:5])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 0 0 0 0]


In [2]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split( X, y, random_state=10, test_size=0.2, stratify=y, shuffle=True )

print(len(X_train))    

120


In [3]:
from torch import nn

import torch.nn.functional as F

#신경망 모형

class Model(nn.Module):

    def __init__(self, input_dim):

        super(Model, self).__init__()

        #input layer, Linear 선형함수(1차함수)

        #input nodes, output nodes 50

        self.layer1 = nn.Linear(input_dim,50) 

        self.layer2 = nn.Linear(50, 20)

        self.layer3 = nn.Linear(20, 3)

        

    def forward(self, x):

        x = F.relu(self.layer1(x))

        x = F.relu(self.layer2(x))

        # 출력층의 활성화함수 - 소프트맥스

        # dim=0 첫번째 차원(row), row 단위로 softmax 적용

        x = F.softmax(self.layer3(x), dim=0) 

        return x

In [4]:
import torch

model = Model(X_train.shape[1]) # 초기화함수의 input_dim으로 변수개수가 전달됨

optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # 최적화함수 정의

loss_fn = nn.CrossEntropyLoss() #손실함수 정의

epochs = 100

In [5]:
#넘파이배열로부터 텐서를 만들고

X_train = torch.from_numpy(X_train).float()

y_train = torch.from_numpy(y_train).long()

for epoch in range(1, epochs+1):

    print("Epoch",epoch)

    #예측값

    y_pred = model(X_train)

    #손실함수에 예측값과 실제값을 입력

    loss = loss_fn(y_pred, y_train)

    print('loss:',loss.item())

    

    # 경사 초기화

    optimizer.zero_grad()

    loss.backward() # 역전파

    optimizer.step() # 가중치 업데이트

Epoch 1
loss: 1.0991902351379395
Epoch 2
loss: 1.0980581045150757
Epoch 3
loss: 1.0969196557998657
Epoch 4
loss: 1.0958075523376465
Epoch 5
loss: 1.0946269035339355
Epoch 6
loss: 1.0932918787002563
Epoch 7
loss: 1.0920765399932861
Epoch 8
loss: 1.0910731554031372
Epoch 9
loss: 1.0902495384216309
Epoch 10
loss: 1.0895804166793823
Epoch 11
loss: 1.0890103578567505
Epoch 12
loss: 1.088532567024231
Epoch 13
loss: 1.088114857673645
Epoch 14
loss: 1.0877271890640259
Epoch 15
loss: 1.0873595476150513
Epoch 16
loss: 1.0870139598846436
Epoch 17
loss: 1.086696982383728
Epoch 18
loss: 1.0864129066467285
Epoch 19
loss: 1.086151361465454
Epoch 20
loss: 1.0859078168869019
Epoch 21
loss: 1.0856553316116333
Epoch 22
loss: 1.0853869915008545
Epoch 23
loss: 1.0851410627365112
Epoch 24
loss: 1.0848994255065918
Epoch 25
loss: 1.084592342376709
Epoch 26
loss: 1.084293007850647
Epoch 27
loss: 1.0840251445770264
Epoch 28
loss: 1.0837804079055786
Epoch 29
loss: 1.0835596323013306
Epoch 30
loss: 1.083360791206

In [6]:
# Prediction

X_test = torch.from_numpy(X_test).float()

pred = model(X_test)

pred[:5]

tensor([[3.7603e-06, 9.5983e-03, 4.2833e-04],
        [6.0092e-02, 5.5357e-06, 1.8325e-11],
        [9.7753e-07, 1.3768e-03, 1.9454e-03],
        [5.4961e-07, 6.4683e-04, 3.0019e-03],
        [9.5056e-05, 2.5962e-01, 5.3586e-06]], grad_fn=<SliceBackward0>)

In [7]:
import numpy as np

np.argmax(pred.data.numpy(), axis=1)

array([1, 0, 2, 2, 1, 2, 0, 2, 2, 0, 0, 1, 2, 2, 1, 0, 0, 1, 2, 0, 2, 2,
       2, 0, 0, 1, 1, 0, 1, 1], dtype=int64)

In [9]:
from sklearn.metrics import accuracy_score

# 모형의 정확도 측정

accuracy_score(y_test, np.argmax(pred.data.numpy(), axis=1))


0.9666666666666667

In [11]:
torch.save(model, "c:/data/iris/iris-torch.h5")

In [12]:
model2 = torch.load("c:/data/iris/iris-torch.h5")

np.argmax(model2(X_test[0]).data.numpy())

2

In [14]:
#pip install torchinfo

from torchinfo import summary

summary(model)

# input_size=(batch size, input nodes)

summary(model, input_size=(32, 4))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [32, 3]                   --
├─Linear: 1-1                            [32, 50]                  250
├─Linear: 1-2                            [32, 20]                  1,020
├─Linear: 1-3                            [32, 3]                   63
Total params: 1,333
Trainable params: 1,333
Non-trainable params: 0
Total mult-adds (M): 0.04
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.01
Estimated Total Size (MB): 0.02